In [56]:
import nbimporter

### Simulator Backend

In [15]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [54]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

In [57]:
from ExpvalCost import ExpvalCost

ham = np.identity(16)
single_cost = ExpvalCost(ham, backend)

NameError: name 'ansatz' is not defined

In [27]:
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]

print(np.identity(4))
print(get_k_basis(2,4))

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [29]:
theta = np.zeros(12)
basis = get_k_basis(4,16)

target_func = ExpvalCost(np.identity(16), backend)
print("target func:")
print(target_func())

target func:


TypeError: __call__() missing 2 required positional arguments: 'params' and 'state_idx'